In [1]:
# conda install -c rapidsai -c nvidia -c conda-forge dask-cuda cudatoolkit=11.6
# nvidia-smi
# conda install -c menpo opencv
from sklearn.model_selection import train_test_split, KFold
import dask.dataframe as dd
import dask.multiprocessing
import dask
import numpy as np
from dask import delayed
import dask_image.imread
import dask_image.ndfilters
import dask_image.ndmeasure
import cv2
from glob import glob
import json
import dask.array as da
import time
import xgboost as xgb
import pandas as pd
import os
from dask import compute
from sklearn.metrics import f1_score
# f1_score(y_true, y_pred, average=[‘micro’, ‘macro’, ‘samples’,’weighted’ 중 하나 선택])
# import tensorflow as tf
from dask.distributed import Client, LocalCluster
from dask_cuda import LocalCUDACluster
import tensorflow as tf

path = os.getcwd()
csv_features = ['내부 온도 1 평균', '내부 온도 1 최고', '내부 온도 1 최저', '내부 습도 1 평균', '내부 습도 1 최고', 
                '내부 습도 1 최저', '내부 이슬점 평균', '내부 이슬점 최고', '내부 이슬점 최저']
# from dask.distributed import Client, LocalCluster
# # client.close()
# cluster = LocalCluster()
# client = Client(cluster)
# print(cluster,client)
# client.close()
# http://localhost:8787/status

# from dask_cuda import LocalCUDACluster
# from dask.distributed import Client
# client.close()
# # http://127.0.0.1:34497/status
# cluster = LocalCUDACluster()
# client = Client(cluster)
# print(cluster,client)

In [2]:
crop = {'1':'딸기','2':'토마토','3':'파프리카','4':'오이','5':'고추','6':'시설포도'}
disease = {'1':{'a1':'딸기잿빛곰팡이병','a2':'딸기흰가루병','b1':'냉해피해','b6':'다량원소결핍 (N)','b7':'다량원소결핍 (P)','b8':'다량원소결핍 (K)'},
           '2':{'a5':'토마토흰가루병','a6':'토마토잿빛곰팡이병','b2':'열과','b3':'칼슘결핍','b6':'다량원소결핍 (N)','b7':'다량원소결핍 (P)','b8':'다량원소결핍 (K)'},
           '3':{'a9':'파프리카흰가루병','a10':'파프리카잘록병','b3':'칼슘결핍','b6':'다량원소결핍 (N)','b7':'다량원소결핍 (P)','b8':'다량원소결핍 (K)'},
           '4':{'a3':'오이노균병','a4':'오이흰가루병','b1':'냉해피해','b6':'다량원소결핍 (N)','b7':'다량원소결핍 (P)','b8':'다량원소결핍 (K)'},
           '5':{'a7':'고추탄저병','a8':'고추흰가루병','b3':'칼슘결핍','b6':'다량원소결핍 (N)','b7':'다량원소결핍 (P)','b8':'다량원소결핍 (K)'},
           '6':{'a11':'시설포도탄저병','a12':'시설포도노균병','b4':'일소피해','b5':'축과병'}}
risk = {'0':'정상','1':'초기','2':'중기','3':'말기'}


#ensemble_labler
label_description = {}
for key, value in disease.items():
    label_description[f'{key}_00_0'] = f'{crop[key]}_정상'
    for disease_code in value:
        for risk_code in risk:
            label = f'{key}_{disease_code}_{risk_code}'
            label_description[label] = f'{crop[key]}_{disease[key][disease_code]}_{risk[risk_code]}'

# label_description = {
# "1_00_0" : "딸기", 
# "2_00_0" : "토마토",
# "2_a5_2" : "토마토_흰가루병_중기",
# "3_00_0" : "파프리카",
# "3_a9_1" : "파프리카_흰가루병_초기",
# "3_a9_2" : "파프리카_흰가루병_중기",
# "3_a9_3" : "파프리카_흰가루병_말기",
# "3_b3_1" : "파프리카_칼슘결핍_초기",
# "3_b6_1" : "파프리카_다량원소결필(N)_초기",
# "3_b7_1" : "파프리카_다량원소결필(P)_초기",
# "3_b8_1" : "파프리카_다량원소결필(K)_초기",
# "4_00_0" : "오이",
# "5_00_0" : "고추",
#  "5_a7_2" : "고추_탄저병_중기",
#  "5_b6_1" : "고추_다량원소결필(N)_초기",
# "5_b7_1" : "고추_다량원소결필(P)_초기",
#  "5_b8_1" : "고추_다량원소결필(K)_초기",
# "6_00_0" : "시설포도",
# "6_a11_1" : "시설포도_탄저병_초기",
#  "6_a11_2" : "시설포도_탄저병_중기",
#  "6_a12_1" : "시설포도_노균병_초기",
# "6_a12_2" : "시설포도_노균병_중기",
#  "6_b4_1" : "시설포도_일소피해_초기",
#  "6_b4_3" : "시설포도_일소피해_말기",
# "6_b5_1" : "시설포도_축과병_초기"   }


global ensemble_label_encoder
ensemble_label_encoder = {key:idx for idx, key in enumerate(label_description)}
ensemble_label_decoder = {val:key for key, val in ensemble_label_encoder.items()}

#crop_labler
crop_label_description = {}
for key, value in disease.items():
    crop_label_description[f'{key}'] = f'{crop[key]}'
    
global crop_label_encoder
crop_label_encoder = {key:idx for idx, key in enumerate(crop_label_description)}
crop_label_decoder = {val:key for key, val in crop_label_encoder.items()}


#disease_labler
disease_label_description = {}
for key, value in disease.items():
    disease_label_description[f'00'] = "정상"
    for disease_code,value in value.items():
        label = f'{disease_code}'
        disease_label_description[label] = f'{value}'

global disease_label_encoder
disease_label_encoder = {key:idx for idx, key in enumerate(disease_label_description)}
disease_label_decoder = {val:key for key, val in disease_label_encoder.items()}


#risk_labler
risk_label_description = {}
for key, value in risk.items():
    label = f'{key}'
    risk_label_description[label] = f'{value}'

global risk_label_encoder
risk_label_encoder = {key:idx for idx, key in enumerate(risk_label_description)}
risk_label_decoder = {val:key for key, val in risk_label_encoder.items()}





In [3]:
crop_label_description

{'1': '딸기', '2': '토마토', '3': '파프리카', '4': '오이', '5': '고추', '6': '시설포도'}

In [4]:
risk_label_description

{'0': '정상', '1': '초기', '2': '중기', '3': '말기'}

In [5]:
disease_label_description

{'00': '정상',
 'a1': '딸기잿빛곰팡이병',
 'a2': '딸기흰가루병',
 'b1': '냉해피해',
 'b6': '다량원소결핍 (N)',
 'b7': '다량원소결핍 (P)',
 'b8': '다량원소결핍 (K)',
 'a5': '토마토흰가루병',
 'a6': '토마토잿빛곰팡이병',
 'b2': '열과',
 'b3': '칼슘결핍',
 'a9': '파프리카흰가루병',
 'a10': '파프리카잘록병',
 'a3': '오이노균병',
 'a4': '오이흰가루병',
 'a7': '고추탄저병',
 'a8': '고추흰가루병',
 'a11': '시설포도탄저병',
 'a12': '시설포도노균병',
 'b4': '일소피해',
 'b5': '축과병'}

In [6]:
label_description

{'1_00_0': '딸기_정상',
 '1_a1_0': '딸기_딸기잿빛곰팡이병_정상',
 '1_a1_1': '딸기_딸기잿빛곰팡이병_초기',
 '1_a1_2': '딸기_딸기잿빛곰팡이병_중기',
 '1_a1_3': '딸기_딸기잿빛곰팡이병_말기',
 '1_a2_0': '딸기_딸기흰가루병_정상',
 '1_a2_1': '딸기_딸기흰가루병_초기',
 '1_a2_2': '딸기_딸기흰가루병_중기',
 '1_a2_3': '딸기_딸기흰가루병_말기',
 '1_b1_0': '딸기_냉해피해_정상',
 '1_b1_1': '딸기_냉해피해_초기',
 '1_b1_2': '딸기_냉해피해_중기',
 '1_b1_3': '딸기_냉해피해_말기',
 '1_b6_0': '딸기_다량원소결핍 (N)_정상',
 '1_b6_1': '딸기_다량원소결핍 (N)_초기',
 '1_b6_2': '딸기_다량원소결핍 (N)_중기',
 '1_b6_3': '딸기_다량원소결핍 (N)_말기',
 '1_b7_0': '딸기_다량원소결핍 (P)_정상',
 '1_b7_1': '딸기_다량원소결핍 (P)_초기',
 '1_b7_2': '딸기_다량원소결핍 (P)_중기',
 '1_b7_3': '딸기_다량원소결핍 (P)_말기',
 '1_b8_0': '딸기_다량원소결핍 (K)_정상',
 '1_b8_1': '딸기_다량원소결핍 (K)_초기',
 '1_b8_2': '딸기_다량원소결핍 (K)_중기',
 '1_b8_3': '딸기_다량원소결핍 (K)_말기',
 '2_00_0': '토마토_정상',
 '2_a5_0': '토마토_토마토흰가루병_정상',
 '2_a5_1': '토마토_토마토흰가루병_초기',
 '2_a5_2': '토마토_토마토흰가루병_중기',
 '2_a5_3': '토마토_토마토흰가루병_말기',
 '2_a6_0': '토마토_토마토잿빛곰팡이병_정상',
 '2_a6_1': '토마토_토마토잿빛곰팡이병_초기',
 '2_a6_2': '토마토_토마토잿빛곰팡이병_중기',
 '2_a6_3': '토마토_토마토잿빛곰팡이병_말기',
 '2_b2_0': '토마토_열과_정상',

In [7]:

@delayed
def padding(data, array_len, col_len):
    pad = np.zeros((array_len, col_len))
    length = min(array_len, len(data))
    
    pad[:length] = data[:length]
    return pad

@delayed
def img_resize(img):
    img = cv2.resize(img, dsize=(224, 224), interpolation=cv2.INTER_AREA)
    img = img.astype(np.float32)/255
    return img


@delayed
def label_encoding(label):
    global ensemble_label_encoder
    encoded_label = ensemble_label_encoder[label]
    return encoded_label

@delayed
def label_encoding_crop(label):
    global crop_label_encoder
    encoded_label = crop_label_encoder[label]
    return encoded_label

@delayed
def label_encoding_disease(label):
    global disease_label_encoder
    encoded_label = disease_label_encoder[label]
    return encoded_label

@delayed
def label_encoding_risk(label):
    global risk_label_encoder
    encoded_label = risk_label_encoder[label]
    return encoded_label

def getlable(jsonpath,type):
    if type == "ensemble":
        with open(jsonpath, 'r') as f:
            json_file = json.load(f)

        crop = json_file['annotations']['crop']
        disease = json_file['annotations']['disease']
        risk = json_file['annotations']['risk']
        label = f'{crop}_{disease}_{risk}'
        return label

    elif type == "crop":
        with open(jsonpath, 'r') as f:
            json_file = json.load(f)
        crop = json_file['annotations']['crop']
        label = f'{crop}'
        return label

    elif type == "dc":
        with open(jsonpath, 'r') as f:
            json_file = json.load(f)
        disease = json_file['annotations']['disease']
        label = f'{disease}'
        return label

    elif type == "risk":  
        with open(jsonpath, 'r') as f:
            json_file = json.load(f)
        risk = json_file['annotations']['risk']
        label = f'{risk}'
        return label

    
def get_label_list(labelpath_list):
    labelarr = np.array([])
    labelarr_crop = np.array([])
    labelarr_dc = np.array([])
    labelarr_risk = np.array([])


    # labelarr = da.array([])
    for ind,json_path in enumerate(labelpath_list):
        # label = label_encoding(getlable(json_path))
        # label = da.array(np.array(label_encoding(getlable(json_path))))
        label = np.array(label_encoding(getlable(json_path,"ensemble")))
        labelarr = np.append(labelarr,label)

        label_crop = np.array(label_encoding_crop(getlable(json_path,"crop")))
        labelarr_crop = np.append(labelarr_crop,label_crop)

        label_dc = np.array(label_encoding_disease(getlable(json_path,"dc")))
        labelarr_dc = np.append(labelarr_dc,label_dc)

        label_risk = np.array(label_encoding_risk(getlable(json_path,"risk")))
        labelarr_risk = np.append(labelarr_risk,label_risk)


    return labelarr, labelarr_crop, labelarr_dc, labelarr_risk

@delayed
def imageresize(img):
    # img = dask.delayed(cv2.resize)(img, dsize=(224, 224), interpolation=cv2.INTER_AREA)
    # img = dask.delayed(img.astype(np.float32)/255)  ##픽셀값을 0~1사이로 정규화
    # # img = np.transpose(img, (2,0,1))
    # return dask.delayed(img.reshape)(-1,224,224,3)
    img = cv2.resize(img, dsize=(224, 224), interpolation=cv2.INTER_AREA)
    img = img.astype(np.float32)/255  ##픽셀값을 0~1사이로 정규화
    # img = np.transpose(img, (2,0,1))
    return np.array(img)

@delayed
def imageresize2(img):
    # img = dask.delayed(cv2.resize)(img, dsize=(224, 224), interpolation=cv2.INTER_AREA)
    # img = dask.delayed(img.astype(np.float32)/255)  ##픽셀값을 0~1사이로 정규화
    # # img = np.transpose(img, (2,0,1))
    # return dask.delayed(img.reshape)(-1,224,224,3)
    img = cv2.resize(img, dsize=(224, 224), interpolation=cv2.INTER_AREA)
    # img = img.astype(np.float32)/255  ##픽셀값을 0~1사이로 정규화
    # img = np.transpose(img, (2,0,1))
    return np.array(img)

In [8]:
@delayed
def get_all_image(imgpath_list,image_size):
    imgarr = np.empty((0,image_size,image_size,3), float)
    for ind,img_path in enumerate(imgpath_list):
        imgarr = delayed(np.append)(imgarr,img, axis = 0)
    return imgarr

* 데이터 준비

In [9]:
%%time
label_list = sorted(glob(os.path.join(path,"data","train","*","*.json")))
y_train,y_train_crop,y_train_dc,y_train_risk, = get_label_list(label_list)
# y_train = y_train.rechunk(5767)
results = dask.compute(*y_train)
label = np.array(results)
# print(label.shape)
results_crop = dask.compute(*y_train_crop)
label_crop = np.array(results_crop)
#
results_dc = dask.compute(*y_train_dc)
label_disease = np.array(results_dc)
#
results_risk = dask.compute(*y_train_risk)
label_risk = np.array(results_risk)
label.shape,label_crop.shape,label_disease.shape,label_risk.shape

Wall time: 32.8 s


((5767,), (5767,), (5767,), (5767,))

In [10]:
# %%time
# #making_csv_set
# row_csv = dd.read_csv(os.path.join(path,"data","train","*","*.csv"),dtype={'외부 누적일사 평균': 'object','내부 이슬점 최고': 'object',
#        '내부 이슬점 최저': 'object',
#        '내부 이슬점 평균': 'object'})
# partitions = row_csv.to_delayed()
# # datas = [padding(part.drop_duplicates(subset=['측정시각'])[csv_features].values,290,9) for part in partitions]
# datas = [padding(part.drop_duplicates(subset=['측정시각'])[csv_features][part != "-"].dropna().values,290,9) for part in partitions[:]]
# X_train = da.array(dask.compute(*datas))


In [11]:
%%time
from dask.distributed import Client, LocalCluster
from dask_cuda import LocalCUDACluster
#http://localhost:3258/status

if __name__ == "__main__":
    with LocalCluster(dashboard_address=':3258') as cluster:
        with Client(cluster) as client:
            print(client,cluster)
            row_csv = dd.read_csv(os.path.join(path,"data","train","*","*.csv"),dtype={'외부 누적일사 평균': 'object','내부 이슬점 최고': 'object',
                '내부 이슬점 최저': 'object',
                '내부 이슬점 평균': 'object'})
            partitions = row_csv.to_delayed()
            # datas = [padding(part.drop_duplicates(subset=['측정시각'])[csv_features].values,290,9) for part in partitions]
            datas = [padding(part.drop_duplicates(subset=['측정시각'])[csv_features][part != "-"].dropna().values,290,9) for part in partitions[:]]
            X_train = da.array(dask.compute(*datas))

<Client: 'tcp://127.0.0.1:52302' processes=4 threads=16, memory=31.41 GiB> LocalCluster(2868fc59, 'tcp://127.0.0.1:52302', workers=4, threads=16, memory=31.41 GiB)
Wall time: 2min 29s


In [12]:
X_train = X_train.reshape(5767,-1)

In [13]:
X_train

dask.array<reshape, shape=(5767, 2610), dtype=float64, chunksize=(5767, 2610), chunktype=numpy.ndarray>

In [14]:
base_model = tf.keras.applications.MobileNetV2(input_shape=(224,224,3),
                                               include_top=False,
                                               weights='imagenet')
base_model.trainable = False
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
model = tf.keras.Sequential([
  base_model,
  global_average_layer,
  
])
model.compile()

In [15]:
@delayed
def imageresize(img):
    # img = dask.delayed(cv2.resize)(img, dsize=(224, 224), interpolation=cv2.INTER_AREA)
    # img = dask.delayed(img.astype(np.float32)/255)  ##픽셀값을 0~1사이로 정규화
    # # img = np.transpose(img, (2,0,1))
    # return dask.delayed(img.reshape)(-1,224,224,3)
    img = cv2.resize(img, dsize=(224, 224), interpolation=cv2.INTER_AREA)
    img = img.astype(np.float32)/255  ##픽셀값을 0~1사이로 정규화
    # img = np.transpose(img, (2,0,1))
    return np.array(img)
#making_img_set
row_img = dask_image.imread.imread(os.path.join(path,"data","train","*","*.jpg"))
train = [imageresize(img) for img in row_img]
train_x = np.array(dask.compute(*train))



In [16]:
%%time
batch = 100
data_length = len(train_x)
loop_num = int(np.floor(data_length/batch))

train_x_feature = dask.array.zeros((data_length,model(train_x[:1]).numpy().shape[1]))


count = 0
for i in range(loop_num):
    print(i)
    if i != loop_num-1:
        start_p = count*batch
        end_p = (count+1)*batch
        train_x_feature[start_p:end_p] = model(train_x[start_p:end_p]).numpy()
        
    else:
        start_p = count*batch
        end_p = (count+1)*batch
        train_x_feature[start_p:] = model(train_x[start_p:]).numpy()
            
    count += 1



0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
Wall time: 1min 48s


In [17]:
train_x_feature

dask.array<setitem, shape=(5767, 1280), dtype=float64, chunksize=(5767, 1280), chunktype=numpy.ndarray>

* csv, feature extraction data 결합

In [18]:
merged_data = da.concatenate((train_x_feature,X_train),axis = 1)

In [19]:
merged_data

dask.array<concatenate, shape=(5767, 3890), dtype=float64, chunksize=(5767, 2610), chunktype=numpy.ndarray>

* train_test split

In [21]:
X, X_test, y, y_test = train_test_split(merged_data, label, test_size=0.2, shuffle= True,stratify=label)

* 모델 학습

In [22]:
%%time
params = {"tree_method": "gpu_hist", 
          "objective": 'multi:softmax',
          "subsample": 0.8,
          "colsample_bytree": 0.8,
        #   "single_precision_histogram" : True,
          }

start = time.time()
XGB_mod1 = xgb.XGBClassifier(params,eval_metric='mlogloss',).fit(X,y)
# print(time.time()-start)
y_pred = XGB_mod1.predict(X_test)

C:\Users\Go\miniconda3\envs\pro1\lib\site-packages\xgboost\core.py:502: FutureWarning: Pass `objective` as keyword args.  Passing these as positional arguments will be considered as error in future releases.
  format(", ".join(args_msg)), FutureWarning
C:\Users\Go\miniconda3\envs\pro1\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


Wall time: 1min 9s


In [23]:
from sklearn.metrics import f1_score
f1_score(y_test, y_pred,average='macro')

0.8814111817616872

In [24]:
from sklearn import metrics

# Print the confusion matrix
print(metrics.confusion_matrix(y_test, y_pred))

# Print the precision and recall, among other metrics
print(metrics.classification_report(y_test, y_pred, digits=3))

[[162   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [  1  28   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [  0   0  38   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [  0   0   0 235   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [  0   0   0   0  20  10   1   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [  0   0   0   0  10  11   1   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [  0   0   0   0   5   2   1   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0  33   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0  24   0   4   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0  31   0   0   0  

In [25]:
answer = np.array([label_description[ensemble_label_decoder[int(val)]] for val in y_test])
predss = np.array([label_description[ensemble_label_decoder[int(val)]] for val in y_pred])

new_crosstab = pd.crosstab(answer, predss, rownames=['answer'], colnames=['preds'])
new_crosstab

preds,고추_고추탄저병_중기,고추_다량원소결핍 (K)_초기,고추_다량원소결핍 (N)_초기,고추_다량원소결핍 (P)_초기,고추_정상,딸기_정상,시설포도_시설포도노균병_중기,시설포도_시설포도노균병_초기,시설포도_시설포도탄저병_중기,시설포도_시설포도탄저병_초기,...,토마토_정상,토마토_토마토흰가루병_중기,파프리카_다량원소결핍 (K)_초기,파프리카_다량원소결핍 (N)_초기,파프리카_다량원소결핍 (P)_초기,파프리카_정상,파프리카_칼슘결핍_초기,파프리카_파프리카흰가루병_말기,파프리카_파프리카흰가루병_중기,파프리카_파프리카흰가루병_초기
answer,,,,,,,,,,,,,,,,,,,,,
고추_고추탄저병_중기,20,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
고추_다량원소결핍 (K)_초기,0,30,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
고추_다량원소결핍 (N)_초기,0,0,29,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
고추_다량원소결핍 (P)_초기,0,0,0,32,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
고추_정상,0,0,0,0,14,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
딸기_정상,0,0,0,0,0,162,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
시설포도_시설포도노균병_중기,0,0,0,0,0,0,6,0,0,0,...,0,0,0,0,0,0,0,0,0,0
시설포도_시설포도노균병_초기,0,0,0,0,0,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
시설포도_시설포도탄저병_중기,0,0,0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,0,0,0


* test셋 예측후 제출

In [20]:
%%time
params = {"tree_method": "gpu_hist", 
          "objective": 'multi:softmax',
          "subsample": 0.8,
          "colsample_bytree": 0.8,
        #   "single_precision_histogram" : True,
          }

start = time.time()
XGB_G = xgb.XGBClassifier(params,eval_metric='mlogloss',).fit(merged_data,label)
# print(time.time()-start)

C:\Users\Go\miniconda3\envs\pro1\lib\site-packages\xgboost\core.py:502: FutureWarning: Pass `objective` as keyword args.  Passing these as positional arguments will be considered as error in future releases.
  format(", ".join(args_msg)), FutureWarning
C:\Users\Go\miniconda3\envs\pro1\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


Wall time: 1min 26s


In [21]:
%%time
print("check")
row_csv = dd.read_csv(os.path.join(path,"data","test","*","*.csv"),dtype={'외부 누적일사 평균': 'object','내부 이슬점 최고': 'object',
    '내부 이슬점 최저': 'object',
    '내부 이슬점 평균': 'object'})
print("check")
partitions = row_csv.to_delayed()
print("check")
# datas = [padding(part.drop_duplicates(subset=['측정시각'])[csv_features].values,290,9) for part in partitions]
datas = [padding(part.drop_duplicates(subset=['측정시각'])[csv_features][part != "-"].dropna().values,290,9) for part in partitions]
print("check")
# X_test = da.array(dask.compute(*datas))

check
check
check
check
Wall time: 53.6 s


In [22]:
@delayed
def imageresize(img):
    # img = dask.delayed(cv2.resize)(img, dsize=(224, 224), interpolation=cv2.INTER_AREA)
    # img = dask.delayed(img.astype(np.float32)/255)  ##픽셀값을 0~1사이로 정규화
    # # img = np.transpose(img, (2,0,1))
    # return dask.delayed(img.reshape)(-1,224,224,3)
    img = cv2.resize(img, dsize=(224, 224), interpolation=cv2.INTER_AREA)
    img = img.astype(np.float32)/255  ##픽셀값을 0~1사이로 정규화
    # img = np.transpose(img, (2,0,1))
    return da.array(img)
#making_img_set
row_img = dask_image.imread.imread(os.path.join(path,"data","test","*","*.jpg"))
test = [imageresize(img) for img in row_img]
# test_x = da.array(dask.compute(*test))



In [23]:
%%time
# os.path.join(path,"sample_submission.csv")
submission = pd.read_csv(os.path.join(path,"sample_submission.csv"))
# submission['label'] = preds
minibatch_size = 200
data_length = len(submission)
loop_num = int(np.floor(data_length/minibatch_size))

# for i 
# xgb.fit(x_prime_train,y_train,xgb_model = xgb)
#http://localhost:3258/status

if __name__ == "__main__":
    with LocalCluster(dashboard_address=':3258') as cluster:
        with Client(cluster) as client:
            print(client,cluster)
            count = 0
            for i in range(loop_num):
                print(i)
                if i !=loop_num-1:
                    start_p = count*minibatch_size
                    end_p = (count+1)*minibatch_size
                    #1
                    temp = da.array(dask.compute(test[start_p:end_p])[0])
                    temp_x_feature = model(temp).numpy()
                    #2
                    temp_x_xgb_feature=da.array(dask.compute(*datas[start_p:end_p])).reshape(len(datas[start_p:end_p]),-1)
                    temp_x_concated = da.concatenate((temp_x_feature,temp_x_xgb_feature),axis = 1)
                    #예측
                    pred_raw = XGB_G.predict(temp_x_concated)
                    submission.loc[start_p:end_p-1,'label'] = np.array([ensemble_label_decoder[int(val)] for val in pred_raw])
                    
                else:
                    start_p = count*minibatch_size
                    #1
                    temp = da.array(dask.compute(test[start_p:])[0])
                    temp_x_feature = model(temp).numpy()
                    #2
                    temp_x_xgb_feature=da.array(dask.compute(*datas[start_p:])).reshape(len(datas[start_p:]),-1)
                    temp_x_concated = da.concatenate((temp_x_feature,temp_x_xgb_feature),axis = 1)
                    #예측
                    pred_raw = XGB_G.predict(temp_x_concated)
                    submission.loc[start_p:,'label'] = np.array([ensemble_label_decoder[int(val)] for val in pred_raw])
                
                count += 1

<Client: 'tcp://127.0.0.1:52489' processes=4 threads=16, memory=31.41 GiB> LocalCluster(939867bb, 'tcp://127.0.0.1:52489', workers=4, threads=16, memory=31.41 GiB)
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
23

In [24]:
submission

,image,label
0,10000,6_00_0
1,10001,5_b6_1
2,10002,4_00_0
3,10003,3_00_0
4,10004,3_b8_1
...,...,...
51901,67673,4_00_0
51902,67674,3_b7_1
51903,67675,6_00_0
51904,67676,2_a5_2


In [25]:

submission.to_csv(os.path.join(path,"gogo_submission_11.csv"), index=False)